In [28]:
from pyshacl import validate
import rdflib
import os

with open("output/wallsTest.ttl", "r") as data_graph_file:
    data_graph_str = data_graph_file.read()


shapes_graph_str = """
@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <https://www.w3.org/2001/XMLSchema#> .
@prefix ex: <http://example.org/myData#> .
@prefix props:  <https://w3id.org/props#> .
@prefix bot:  <https://w3id.org/bot#> .

ex:WallShape
    a sh:NodeShape ;
    sh:targetClass ex:walls ;
    sh:property [
        sh:path props:Thermaltransmittance ;
        sh:datatype xsd:double ;
                sh:minCount 1 ;
                sh:maxCount 1 ;
                sh:maxInclusive 0.11 ;
                sh:message bot:hasGuid
    ] .
"""

shapes_graph_str_sparql = '''
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix props: <https://w3id.org/props#> .
@prefix ex: <http://example.org/myData#> .
@prefix bot: <https://w3id.org/bot#> .

ex:WallShape
    a sh:NodeShape ;
    sh:targetClass ex:walls ;
    sh:sparql [
        a sh:SPARQLConstraint ;
        sh:message "Thermal transmittance exceeds the limit for wall with GUID: {$guid}. Result: {$result}. Value: {$transmittance}" ;
        sh:prefixes [
            sh:declare [
                sh:prefix "props" ;
                sh:namespace "https://w3id.org/props#"^^xsd:anyURI ;
            ] ;
            sh:declare [
                sh:prefix "bot" ;
                sh:namespace "https://w3id.org/bot#"^^xsd:anyURI ;
            ] ;
        ] ;
        sh:select """
            SELECT ?this ?guid ?result ?transmittance WHERE {
                ?this bot:hasGuid ?guid .
                ?this props:Thermaltransmittance ?transmittance .
                BIND(IF(?transmittance > 0.11, "False", "True") AS ?result)
            }
        """ ;
    ] .
'''

# Load your data graph from string
data_graph = rdflib.Graph()
data_graph.parse(data=data_graph_str, format="turtle")
#data_graph.print()

# Load your SHACL shapes graph from string
shapes_graph = rdflib.Graph()
shapes_graph.parse(data=shapes_graph_str_sparql, format="turtle")

# Validate the data graph against the shapes graph
conforms, results_graph, results_text = validate(
    data_graph,
    shacl_graph=shapes_graph,
    ont_graph=None,
    inference='rdfs',  # or 'owlrl', 'none'
    abort_on_first=False,
    meta_shacl=False,
    advanced=True,
    debug=True
)

# Print the results
print("Conforms:", conforms)
print(results_text)

# Optionally, you can serialize the results graph to a file
results_graph.serialize("output/validation_report.ttl", format="turtle")


Cloning DataGraph to temporary memory graph before pre-inferencing.
Running pre-inferencing with option='owlrl'.
Found 1 SHACL Shapes defined with type sh:NodeShape.
Found 0 SHACL Shapes defined with type sh:PropertyShape.
Found 0 property paths to follow.
Found 1 implied SHACL Shapes based on their properties.
Found 0 implied SHACL Shapes used as values in shape-expecting constraints.
Cached 1 unique NodeShapes and 0 unique PropertyShapes.
Activating SHACL-AF Features.
Will run validation on 1 named graph/s.
Validating DataGraph named N634bb051cc964661a35452771bc5d45f
Checking if Shape <NodeShape http://example.org/myData#WallShape> defines its own targets.
Identifying targets to find focus nodes.
Milliseconds to find focus nodes: 0.240ms
Found 76 Focus Nodes to evaluate.
Running evaluation of Shape <NodeShape http://example.org/myData#WallShape>
Current shape evaluation path: <NodeShape http://example.org/myData#WallShape>
Checking conformance for constraint: <SPARQLBasedConstraint o

Conforms: False
Validation Report
Conforms: False
Results (76):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: ex:WallShape
	Focus Node: inst:wall_406
	Value Node: inst:wall_406
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:message Literal("Thermal transmittance exceeds the limit for wall with GUID: {$guid}. Result: {$result}. Value: {$transmittance}") ; sh:prefixes [ sh:declare [ sh:namespace Literal("https://w3id.org/bot#", datatype=xsd:anyURI) ; sh:prefix Literal("bot") ], [ sh:namespace Literal("https://w3id.org/props#", datatype=xsd:anyURI) ; sh:prefix Literal("props") ] ] ; sh:select Literal("
            SELECT ?this ?guid ?result ?transmittance WHERE {
                ?this bot:hasGuid ?guid .
                ?this props:Thermaltransmittance ?transmittance .
                BIND(IF(?transmittance > 0.11, "False", "True") AS ?result)
            }
        ") ]
	Message: Th

<Graph identifier=N628e59d5319c4e649dcf0f9e8de79836 (<class 'rdflib.graph.Graph'>)>